# COVID-19 Data Extraction from Worldometers Web

Extract the World covid-19 dataset and USA dataset by each state

Data Source: 
Worldometers - https://www.worldometers.info/coronavirus/#countries

Run the Commands to get latest dataset

pip install -r requirements.txt python covid19_data_scrap.py

World dataset is extracted and stored in AWS RD MySql table called covid_world.

USA dataset is extracted and stored in the AWS RD MySQL table called covid_usa.

# Import Python Libraries

In [6]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import sqlalchemy

# Connect to AWS MySql

In [7]:
#connect to MySQL DB in AWS
db_username = 'xxxx'
db_password = 'xxxxx'
db_ip       = 'covid19db.xxxxxx.rds.amazonaws.com'
db_name     = 'xxxx'
db_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                         format(db_username, db_password, db_ip, db_name))

# Parse dataset and convert to JSON

In [10]:
#get url for world data
res = requests.get("https://www.worldometers.info/coronavirus/")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

# to json
a = df[0].to_json(orient='records')
print(a)

[{"Country,Other":"World","TotalCases":2405719,"NewCases":"+74,951","TotalDeaths":164952.0,"NewDeaths":"+4,905","TotalRecovered":624911.0,"ActiveCases":1615856,"Serious,Critical":54225.0,"Tot\u00a0Cases\/1M pop":309.0,"Deaths\/1M pop":21.2,"TotalTests":null,"Tests\/ 1M pop":null},{"Country,Other":"USA","TotalCases":764447,"NewCases":"+25,655","TotalDeaths":40554.0,"NewDeaths":"+1,540","TotalRecovered":71001.0,"ActiveCases":652892,"Serious,Critical":13566.0,"Tot\u00a0Cases\/1M pop":2309.0,"Deaths\/1M pop":123.0,"TotalTests":3861382.0,"Tests\/ 1M pop":11666.0},{"Country,Other":"Spain","TotalCases":198674,"NewCases":"+4,258","TotalDeaths":20453.0,"NewDeaths":"+410","TotalRecovered":77357.0,"ActiveCases":100864,"Serious,Critical":7371.0,"Tot\u00a0Cases\/1M pop":4249.0,"Deaths\/1M pop":437.0,"TotalTests":930230.0,"Tests\/ 1M pop":19896.0},{"Country,Other":"Italy","TotalCases":178972,"NewCases":"+3,047","TotalDeaths":23660.0,"NewDeaths":"+433","TotalRecovered":47055.0,"ActiveCases":108257,"S

# JSON to DataFrame

In [11]:
df1 = pd.read_json(a,orient='columns')
print(df1)

             Country,Other  TotalCases NewCases  TotalDeaths NewDeaths  \
0                    World     2405719  +74,951     164952.0    +4,905   
1                      USA      764447  +25,655      40554.0    +1,540   
2                    Spain      198674   +4,258      20453.0      +410   
3                    Italy      178972   +3,047      23660.0      +433   
4                   France      152894   +1,101      19718.0      +395   
..                     ...         ...      ...          ...       ...   
209               Anguilla           3     None          NaN      None   
210  Saint Pierre Miquelon           1     None          NaN      None   
211                  Yemen           1     None          NaN      None   
212                  China       82735      +16       4632.0      None   
213                 Total:     2405719  +74,951     164952.0    +4,905   

     TotalRecovered  ActiveCases  Serious,Critical  Tot Cases/1M pop  \
0          624911.0      1615856       

# Load to MySQL DB

In [ ]:
#Data from DataFrams is loaded into MYSql DB in AWS RD.
df1.to_sql(con=db_connection, name='covid_world', if_exists='replace')